In [190]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df1 = pd.read_csv("casos.csv")
df2 = pd.read_csv("gordon.csv")

In [191]:
df1.estado_caso.value_counts()

3    14
2    10
1     8
Name: estado_caso, dtype: int64

In [192]:
df2.head()

,id_caso,respuesta
0,100,1
1,101,0
2,102,0
3,103,1
4,104,1


In [193]:
# Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando
# aquellos casos en los que no participó Batman).

    
def contar_n(serie, n):

    serie = serie.apply(lambda x: 1 if x==n else 0)#.apply(hacer_dos)
    return np.sum(serie)

def contar_distinto_a_n(serie, n):

    serie = serie.apply(lambda x: 1 if x!=n else 0)#.apply(hacer_dos)
    return np.sum(serie)

#Cuento los que tienen estado 2
df3 = df1.groupby('categoria')['estado_caso'].agg(lambda x: contar_n(x, 2)).reset_index()

#Cuento los que tienen estado distinto de 2
#df4 = df1.groupby('categoria')['estado_caso'].agg(lambda x: contar_distinto_a_n(x, 2)).reset_index()
#NO ERA NECESARIO, ME INTERESABAN LOS TOTALES

#Cuento todos los casos
df4 = df1.groupby('categoria').agg({'estado_caso' : 'count'}).reset_index()

In [194]:
df3.rename(columns = {'estado_caso' : 'casos_resueltos'}, inplace = True)
df3.head(10)

,categoria,casos_resueltos
0,cat1,1
1,cat2,2
2,cat3,1
3,cat4,4
4,cat5,1
5,cat6,1
6,cat7,0
7,cat8,0


In [195]:
df4.rename(columns = {'estado_caso' : 'casos_totales'}, inplace = True)
df4.head(10)

,categoria,casos_totales
0,cat1,2
1,cat2,3
2,cat3,5
3,cat4,7
4,cat5,6
5,cat6,5
6,cat7,3
7,cat8,1


In [196]:
#Hago un join de los dos dataframes

tasa_casos = pd.merge(df4, df3, on = 'categoria', how = 'inner')
tasa_casos.head(10)

,categoria,casos_totales,casos_resueltos
0,cat1,2,1
1,cat2,3,2
2,cat3,5,1
3,cat4,7,4
4,cat5,6,1
5,cat6,5,1
6,cat7,3,0
7,cat8,1,0


In [197]:
tasa_casos['tasa de resolucion'] = tasa_casos['casos_resueltos'] / tasa_casos['casos_totales'] * 100
tasa_casos.head()

,categoria,casos_totales,casos_resueltos,tasa de resolucion
0,cat1,2,1,50.000000
1,cat2,3,2,66.666667
2,cat3,5,1,20.000000
3,cat4,7,4,57.142857
4,cat5,6,1,16.666667


In [198]:
# Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos
# en los que fue llamado con la batiseñal, participó en la resolución)
# Es el mismo procedimiento pero debo filtrar antes aquellos donde no ayudo batman

casos_batman = pd.merge(df1, df2, on = 'id_caso', how = 'inner')
casos_batman = casos_batman[casos_batman['respuesta'] == 1]
casos_batman.head()

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud,respuesta
0,08/08/19,100,aca iria una desc,1,cat1,34°,35°,1
3,05/08/19,103,aca iria otra desc,3,cat4,35°,36°,1
4,07/08/19,104,aca iria una desc,2,cat1,33°,34°,1
6,05/09/19,106,aca iria una desc,2,cat4,36°,37°,1
8,07/09/19,108,aca iria una desc,2,cat3,-32°,-33°,1


In [199]:
#Cuento todos los casos en los que participo que tienen estado 2
df5 = casos_batman.groupby('categoria')['estado_caso'].agg(lambda x: contar_n(x, 2)).reset_index()

#Cuento todos los casos en los que participo
df6 = casos_batman.groupby('categoria').agg({'estado_caso' : 'count'}).reset_index()

In [200]:
df5.rename(columns = {'estado_caso' : 'casos_resueltos'}, inplace = True)
df5.head(10)

,categoria,casos_resueltos
0,cat1,1
1,cat2,1
2,cat3,1
3,cat4,3
4,cat5,1
5,cat6,1
6,cat7,0


In [201]:
df6.rename(columns = {'estado_caso' : 'casos_totales'}, inplace = True)
df6.head(10)

,categoria,casos_totales
0,cat1,2
1,cat2,1
2,cat3,2
3,cat4,5
4,cat5,3
5,cat6,3
6,cat7,2


In [202]:
#Hago un join de los dos dataframes

casos_batman = pd.merge(df5, df6, on = 'categoria', how = 'inner')
casos_batman.head(10)

,categoria,casos_resueltos,casos_totales
0,cat1,1,2
1,cat2,1,1
2,cat3,1,2
3,cat4,3,5
4,cat5,1,3
5,cat6,1,3
6,cat7,0,2


In [204]:
#Hago un join de los dos dataframes

casos_batman['tasa de res con batman'] = casos_batman['casos_resueltos'] / casos_batman['casos_totales'] * 100
casos_batman.head(8)

,categoria,casos_resueltos,casos_totales,tasa de res con batman
0,cat1,1,2,50.000000
1,cat2,1,1,100.000000
2,cat3,1,2,50.000000
3,cat4,3,5,60.000000
4,cat5,1,3,33.333333
5,cat6,1,3,33.333333
6,cat7,0,2,0.000000
